### Autograd в PyTorch

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

При обучении нейронных сетей мы минимизируем значение функции ошибки на обучающей выборке, меняя значения параметров модели. Чтобы понять, куда нужно сместить значения параметров, нужно уметь считать градиент — именно для автоматизации этих расчётов нам и нужен Pytorch. Посмотрим, как именно он это делает.

Но перед этим вспомним правило дифференцирования сложной функции:

$
\begin{align}
\frac{\partial f}{\partial x} =
\frac{\partial f}{\partial u} \cdot 
\frac{\partial u}{\partial x}
\end{align}
$

Пример:

$$
f(x) = \sin(\ln x) \quad
u(x) = \ln x  \quad
f(u) = \sin(u)$$

$$
\frac{\partial f}{\partial x} =
\frac{\partial f}{\partial u} \cdot 
\frac{\partial u}{\partial x} = 
\cos(u) \cdot 
\frac{1}{x} = 
\cos(\ln x) \cdot 
\frac{1}{x}
$$

#### 1. Дифференцирование и вычислительный граф

Рассмотрим выражение $f(x, y) = x^2 + xy + (x + y)^2$ и построим его граф:

<img src="../assets/images/forward_pass.png" style="background:white" width="300"/>

Производная по переменной $x$:
$$
\begin{align*}
\frac{\partial f}{\partial x} &=
\textcolor{violet}{
\frac{\partial f}{\partial d} \cdot 
\frac{\partial d}{\partial a} \cdot 
\frac{\partial a}{\partial x}
}
+
\textcolor{teal}{
\frac{\partial f}{\partial d} \cdot 
\frac{\partial d}{\partial b} \cdot 
\frac{\partial b}{\partial x}
}
+
\textcolor{orange}{
\frac{\partial f}{\partial e} \cdot 
\frac{\partial e}{\partial c} \cdot 
\frac{\partial c}{\partial x}
} \\
&=
\textcolor{violet}{
1 \cdot 1 \cdot 2x
}
+
\textcolor{teal}{
1 \cdot 1 \cdot y
}
+
\textcolor{orange}{
1 \cdot 2c \cdot 1
}
=
2x + y + 2(x+y)
\end{align*}
$$

Для вычисления производных Pytorch строит вычислительный граф, проход по которому позволяет рассчитать градиенты по правилу производной сложной функции (chain rule).

Прямой проход:
- расчёт значения выходного тензора
- построение графа и сохранение нужных для обратного прохода данных для каждой операции

Обратный проход (вызов `.backward()` у корня графа):
- расчёт градиентов и их накопление в артибуте `.grad` каждого тензора
- распространение вычислений далее до листьев графа

Запишем выражение для $f(x, y)$, задав начальные условия $x = 2.0, y = 2.0$.

In [1]:
import torch

In [2]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
a = x**2
b = x * y
c = x + y
d = a + b
e = c**2
f = d + e
f

tensor(24., grad_fn=<AddBackward0>)

`grad_fn` означает, что `f` не просто отдельный тензор, а связан с вычислительным графом и соответствует операции `Add`

Запустим backprop и убедимся, что градиенты рассчитаны правильно:

$\frac{\partial f}{\partial x} = 2x + y + 2(x + y)$

$\frac{\partial f}{\partial y} = x + 2(x + y)$

In [3]:
f.backward()
print(x.grad)
print(y.grad)

tensor(14.)
tensor(10.)


Повторный вызов `backward()` приведёт к ошибке, потому что после предыдущего вызова граф уже уничтожен для высвобождения ресурсов. Такое поведение по умолчанию оправдано, но если мы по какой-то причине хотим сохранить граф, следует указать `.backward(retain_graph=True)`

In [4]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

f = x**2 + x * y + (x + y)**2

# вызовем backward дважды и посмотрим на градиенты:
print("Первый вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)

print("\nВторой вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)

Первый вызов
Производная по x:  tensor(14.)
Производная по x:  tensor(10.)

Второй вызов
Производная по x:  tensor(28.)
Производная по x:  tensor(20.)


После второго вызова градиенты удвоились, но здесь нет ошибки: градиенты накапливаются в поле `.grad`, и если мы хотим избавиться от истории прошлых вычислений, это стоит сделать явно

In [5]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
a = x**2
b = x * y
c = x + y
d = a + b
e = c**2
f = d + e

# вызовем backward дважды и посмотрим на градиенты:
print("Первый вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)
# обнулим градиенты, можно сделать двумя способами
x.grad = None
y.grad = torch.tensor(0.0)

print("\nВторой вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)

Первый вызов
Производная по x:  tensor(14.)
Производная по x:  tensor(10.)

Второй вызов
Производная по x:  tensor(14.)
Производная по x:  tensor(10.)


Иногда мы можем захотеть посмотреть на градиенты не только в листовых вершинах графа, но и во внутренних. По умолчанию Pytorch эти данные не сохраняет:

In [6]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
z = y**2
z.backward()
y.grad

/var/folders/gz/zq5l18116xzf83g88xv7jqnr0000gp/T/ipykernel_62713/541772334.py:5: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  y.grad


Но если очень нужно, то мы можем пометить любой тензор, где хочется узнать значение производной, вызвав метод `.retain_grad()`

In [7]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
# помечаем y
y.retain_grad()
z = y**2
z.backward(retain_graph=True)
y.grad

tensor(4.)

#### 2. Отключение расчёта градиентов

https://pytorch.org/docs/stable/notes/autograd.html#locally-disable-grad-doc

Иногда мы хотим локально отключить расчёт градиентов: это может пригодится, когда мы хотим изменить значения каких-то параметров и не сломать при этом вычислительный граф (о том, что может происходить, если в нужном месте не отключить трекинг градиентов, см ниже в разделе $2.3$)

Несколько способов:
1. Изменить значение атрибута тензора `requires_grad` напрямую

In [8]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
print(y.requires_grad)  # True
x.requires_grad = False
y = x + 1
print(y.requires_grad)  # False

True
False


2. Использовать `torch.no_grad()` (как менеджер контекста или как декоратор)

In [9]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
print(x.requires_grad)  # True

# локально отключаем трекинг градиентов
with torch.no_grad():
    y = x + 1
print(y.requires_grad)  # False

True
False


In [10]:
# декоратор
def add_one(t: torch.Tensor) -> torch.Tensor:
    return t + 1


@torch.no_grad()
def add_two(t: torch.Tensor) -> torch.Tensor:
    return t + 1


x = torch.tensor(1.0, requires_grad=True)
y = add_one(x)
print(y.requires_grad)  # True
z = add_two(x)
print(z.requires_grad)  # False

True
False


3. Получить копию тензора с помощью метода `.detach()`

In [11]:
x = torch.tensor(1.0, requires_grad=True)
y = x + 1
print(x.requires_grad)  # True
z = y.detach()
print(z.requires_grad)  # False

True
False


#### 3. Некоторые распространённые ошибки

1. Обновление параметра на месте вне контекста `torch.no_grad()` вызовет ошибку. PyTorch такое явно запрещает

In [12]:
w = torch.tensor(1.0, requires_grad=True)
# x = torch.tensor(1.0)
f = w + 1
f.backward()
w -= w.grad

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

2. Обновление параметра через присваивание приводит к тому, что параметр больше не является листом графа, и следующая итерация приведёт к ошибке:

In [13]:
w = torch.tensor(1.0, requires_grad=True)
# x = torch.tensor(1.0)
f = w + 1
# первая итерация - всё как будто ок
f.backward()
w = w - w.grad  # на этом моменте `w` - больше не лист графа
f = w + 1

# вторая итерация - оказывается, что не ок
f.backward()
w = w - w.grad  # здесь w.grad is None!

/var/folders/gz/zq5l18116xzf83g88xv7jqnr0000gp/T/ipykernel_62713/963600647.py:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  w = w - w.grad  # здесь w.grad is None!


TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

3. Внутри контекста `torch.no_grad()` параметр обновляем не на месте, а через переназначение, после этого он более не ожидает градиентов, всё ломается при вызове `.backward()`

In [14]:
w = torch.tensor(1.0, requires_grad=True)
f = w + 1
# первая итерация
f.backward()
with torch.no_grad():
    w = w - w.grad  # упс, для w теперь requires_grad = False!
f = w + 1

# вторая итерация
f.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn